In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Mean-Variance Optimizer
def mean_variance_optimizer(portfolio, target_dts_value, risk_free_rate):
    """
    Optimize portfolio weights to maximize weighted excess return while meeting target DTS.

    Parameters:
    - portfolio: pd.DataFrame
        Includes columns: 'DTS', 'ytm' 
    - target_dts_value: float
    - risk_free_rate: float

    Returns:
    - np.ndarray: Optimized weights if successful, else None.
    """
    n = len(portfolio)

    # Objective function: maximize portfolio excess return
    def objective(weights):
        excess_returns = portfolio['ytm'] - risk_free_rate
        return -np.dot(weights, excess_returns)  # Negative because we minimize

    # Constraints
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.dot(w, portfolio['DTS']) - target_dts_value},  # Target DTS
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # Weights sum to 1
    ]

    # Bounds for weights: [0, 1]
    bounds = [(0, 1) for _ in range(n)]

    # Initial guess: Equal weights
    initial_weights = np.ones(n) / n

    # Perform optimization
    result = minimize(objective, initial_weights, bounds=bounds, constraints=constraints)
    return result.x if result.success else None

In [ ]:
# Hedging Function
def hedge_duration(portfolio_duration, treasury_data):
    """
    Hedge portfolio duration using Treasury 

    Parameters:
    - portfolio_duration: float
        Total duration of the portfolio.
    - treasury_data: pd.DataFrame
        Includes columns: 'treasury_duration', 'price', 'available_quantity'.

    Returns:
    - pd.DataFrame: Details of the Treasury instruments used for hedging.
    """
    remaining_duration = portfolio_duration
    hedge_results = []

    # Sort Treasury data by descending duration
    treasury_data = treasury_data.sort_values(by='treasury_duration', ascending=False)

    for _, treasury in treasury_data.iterrows():
        tool_duration = treasury['treasury_duration']
        max_quantity = treasury['available_quantity']

        # Quantity needed to hedge (bounded by available quantity)
        needed_quantity = min(abs(remaining_duration / tool_duration), max_quantity)

        # Update remaining duration
        remaining_duration += needed_quantity * tool_duration

        # Record hedging details
        hedge_results.append({
            'treasury_tool': treasury.name,
            'used_quantity': needed_quantity,
            'treasury_duration': tool_duration,
            'cost': needed_quantity * treasury['price']  # Cost of the hedge
        })

        # Stop if fully hedged
        if abs(remaining_duration) < 1e-6:  # Tolerance
            break

    # Return results as a DataFrame
    return pd.DataFrame(hedge_results)



In [ ]:
# Portfolio Return Calculation
def calculate_portfolio_return(sub_data, risk_free_rate, treasury_hedge):
    """
    Calculate portfolio returns including coupon income, capital gains, and hedging costs.

    Parameters:
    - sub_data: pd.DataFrame
        Includes columns: 'Weight', 'coupon_rate', 'price', 'prev_price'.
    - risk_free_rate: float
        Risk-free rate for excess return calculation.
    - treasury_hedge: pd.DataFrame
        Hedging results containing 'cost'.

    Returns:
    - dict: Portfolio return components and metrics.
    """
    # Coupon income
    coupon_income = (sub_data['Weight'] * sub_data['coupon_rate'] * 100).sum()  # Face value = 100

    # Capital gains
    capital_gains = (sub_data['Weight'] * (sub_data['price'] - sub_data['prev_price'])).sum()

    # Hedge cost
    hedge_cost = treasury_hedge['cost'].sum()

    # Portfolio cost (total invested amount)
    portfolio_cost = (sub_data['Weight'] * sub_data['price']).sum()

    # Total return
    total_return = coupon_income + capital_gains - hedge_cost

    # Excess return
    excess_return = total_return - risk_free_rate * portfolio_cost

    return {
        'coupon_income': coupon_income,
        'capital_gains': capital_gains,
        'hedge_cost': hedge_cost,
        'portfolio_cost': portfolio_cost,
        'total_return': total_return,
        'excess_return': excess_return
    }

In [ ]:
# Trading and Hedging Framework
def trading_and_hedging(trading_data, treasury_data, interest_rate_data, target_dts):
    """
    Perform trading and hedging operations, calculate portfolio metrics dynamically.

    Parameters:
    - trading_data: pd.DataFrame
    - treasury_data: pd.DataFrame
    - interest_rate_data: pd.DataFrame
    - target_dts: dict

    Returns:
    - pd.DataFrame: Results of trading, hedging, and performance metrics.
    """
    results = []

    # Calculate previous prices
    trading_data = trading_data.sort_values(by=['CUSIP', 'Date'])
    trading_data['prev_price'] = trading_data.groupby('CUSIP')['price'].shift(1)

    for date, daily_data in trading_data.groupby('Date'):
        for subportfolio, sub_data in daily_data.groupby('subportfolio'):
            
            # Get Treasury data for the current date
            daily_treasury_data = treasury_data[treasury_data['Date'] == date]

            # Get risk-free rate for the current date
            interest_rate_row = interest_rate_data[interest_rate_data['Date'] == date]
            risk_free_rate = interest_rate_row['rate'].iloc[0]

            # Get target DTS for the current subportfolio
            target_dts_value = target_dts.get(subportfolio, None)

            # Optimize portfolio weights
            optimized_weights = mean_variance_optimizer(sub_data, target_dts_value, risk_free_rate)
            if optimized_weights is not None:
                sub_data['Weight'] = optimized_weights

                # Calculate portfolio duration for hedging
                portfolio_duration = (sub_data['Weight'] * sub_data['modified_duration']).sum()
                treasury_hedge = hedge_duration(portfolio_duration, daily_treasury_data)

                # Calculate portfolio returns
                return_metrics = calculate_portfolio_return(
                    sub_data=sub_data,
                    risk_free_rate=risk_free_rate,
                    treasury_hedge=treasury_hedge
                )

                # Record results
                results.append({
                    'date': date,
                    'subportfolio': subportfolio,
                    **return_metrics
                })

    return pd.DataFrame(results)

In [ ]:
if __name__ == "__main__":
    # Example Input Data Preparation
    # 1. Trading Data
    trading_data = pd.DataFrame({
        'Date': ['2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02'], # I need trading + 1day for the shift(1)
        'CUSIP': ['123456', '654321', '123456', '654321'],
        'DTS': [100, 200, 110, 210],
        'modified_duration': [5.0, 6.0, 5.2, 6.1],
        'ytm': [0.03, 0.05, 0.031, 0.051],
        'price': [100, 95, 101, 96],
        'coupon_rate': [0.04, 0.03, 0.04, 0.03],
        'subportfolio': ['high', 'medium', 'high', 'medium']
    })

    # 2. Treasury Data
    treasury_data = pd.DataFrame({
        'Date': ['2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02'],
        'treasury_duration': [3.0, 7.0, 3.1, 7.1],
        'price': [102, 98, 103, 99],
        'available_quantity': [50, 30, 50, 30]
    })

    # 3. Interest Rate Data
    interest_rate_data = pd.DataFrame({
        'Date': ['2023-01-01', '2023-01-02'],
        'rate': [0.02, 0.022]
    })

    # 4. Target DTS
    target_dts = {
        'high': 200,  # Target DTS for the high-risk subportfolio
        'medium': 150,  # Target DTS for the medium-risk subportfolio
        'low': 100      # Target DTS for the low-risk subportfolio
    }

    # Call Trading and Hedging Framework
    results = trading_and_hedging(
        trading_data=trading_data,
        treasury_data=treasury_data,
        interest_rate_data=interest_rate_data,
        target_dts=target_dts
    )

    # Print Results
    print("Trading and Hedging Results:")
    print(results)
